# Constants and Classes

In [1]:
# Constants
default_skill_value = 0 # value for a players skill if no player_skill is provided on player creation
skill_value_modifier = 0.03
total_of_decks = 11 # total amount of decks
player_num = 512 # total number of players in a tournament
num_rounds = 14 # total number of rounds in a tournament



# Player Tournament and classes
import numpy as np
import pandas as pd
from threading import Lock

class Deck:
    def __init__(self, name):
        self.name = name
        self.matchup_spread = {}

    def set_matchup_win_prob(self, opponent_deck_name, win_prob):
        self.matchup_spread[opponent_deck_name] = win_prob

    def get_matchup_win_prob(self, opponent_deck_name):
        return self.matchup_spread.get(opponent_deck_name, 0.5)
    
    def __str__(self):
        return self.name
    
    def __reduce__(self):
        # Return the class itself, arguments, and the state
        return (self.__class__, (self.name,), {'name': self.name, 'matchup_spread': self.matchup_spread})
    
    def __setstate__(self, state):
        # Set the object's state from the given state dictionary
        self.__dict__.update(state)

class DeckManager:
    def __init__(self):
        self.decks = {}

    def add_deck(self, deck):
        self.decks[deck.name] = deck

    def generate_win_probabilities(self, deck_names):
        num_decks = len(deck_names)
        for i in range(num_decks):
            for j in range(i + 1, num_decks):
                win_prob = np.clip(np.random.normal(0.5, 0.15), 0, 1)
                self.decks[deck_names[i]].set_matchup_win_prob(deck_names[j], win_prob)
                self.decks[deck_names[j]].set_matchup_win_prob(deck_names[i], 1 - win_prob)

    def get_win_prob_matrix(self):
        deck_names = list(self.decks.keys())
        num_decks = len(deck_names)
        matrix = np.zeros((num_decks, num_decks))
        for i in range(num_decks):
            for j in range(num_decks):
                matrix[i, j] = self.decks[deck_names[i]].get_matchup_win_prob(deck_names[j])
        return matrix

    def get_win_prob_dataframe(self):
        deck_names = list(self.decks.keys())
        num_decks = len(deck_names)
        matrix = np.zeros((num_decks, num_decks))
        for i in range(num_decks):
            for j in range(num_decks):
                matrix[i, j] = self.decks[deck_names[i]].get_matchup_win_prob(deck_names[j])
        df = pd.DataFrame(matrix, index=deck_names, columns=deck_names)
        return df

    def load_win_probabilities_from_csv(self, file_path):
        # Read the Excel file
        df = pd.read_csv(file_path, header=0, index_col=0)
        
        # Drop the Representation column if it exists
        if 'Representation' in df.columns:
            df = df.drop(columns=['Representation'])
        
        # Ensure decks are added to the manager
        deck_names = df.index.to_list()
        for deck_name in deck_names:
            if deck_name not in self.decks:
                self.add_deck(Deck(deck_name))
        
        # Populate the matchup probabilities
        for i, row in df.iterrows():
            for j, value in row.items():
                self.decks[i].set_matchup_win_prob(j, value)
    
    def __reduce__(self):
        # Return the class itself, no arguments, and the state
        return (self.__class__, (), {'decks': self.decks})
    
    def __setstate__(self, state):
        # Set the object's state from the given state dictionary
        self.__dict__.update(state)

class Player:
    def __init__(self, player_id, alias, player_skill=default_skill_value, deck=None):
        self.id = player_id
        self.alias = alias
        self.skill = player_skill
        self.deck = deck
        self.wins = 0
        self.losses = 0
        self.history = []  # List to store match results
        self.history_lock = Lock()
    
    def __reduce__(self):
        # The object's state is returned as a tuple:
        # (callable, arguments_to_callable, additional_state)
        # Lock object is not pickled, so we're not including it in the state.
        return (self.__class__, (self.id, self.alias, self.skill, self.deck), {'wins': self.wins, 'losses': self.losses, 'history': self.history})
    
    def __setstate__(self, state):
        self.wins = state.get('wins', 0)
        self.losses = state.get('losses', 0)
        self.history = state.get('history', [])
        self.history_lock = Lock()  # Initialize a new Lock object after unpickling

    def set_deck(self, deck):
        """Set the deck for the player."""
        self.deck = deck

    def get_deck(self):
        """Retrieve the player's deck."""
        return self.deck

    def record_match(self, tournament, tournament_round, opponent, result):
        with self.history_lock:
            self.history.append([tournament.name, tournament_round, str(opponent.id) + opponent.alias, result])
            
    def get_tournament_history(self, tournament):
        tournament_results = [history for history in self.history if history[0] == tournament.name]
        return tournament_results

    def get_tournament_standing(self, tournament):
        tournament_results = [history for history in self.history if history[0] == tournament.name]
        wins = 0
        losses = 0
        for match in tournament_results:
            result = match[-1]
            if result == "W":
                wins += 1
            elif result == "L":
                losses += 1
        return (wins, losses)

    def __str__(self):
        return f"[{self.id}] {self.alias} ({self.get_deck().name}) - Skill Level: {self.skill}"

    def __repr__(self):
        return f"[{self.id}] {self.alias} ({self.get_deck().name}) - Skill Level: {self.skill}"


# Generate Deck Win Prob Matrix

In [3]:
# Example setup
from names import get_first_name
from random import sample, choice

deck_manager = DeckManager()
deck_manager.load_win_probabilities_from_csv('metagame_spread.csv')

import pickle
# Pickle the list of deck matchup spread
with open("deck_manager.pkl", "wb") as f:
    pickle.dump(deck_manager, f)

# Generate Player Win Prob Matrix

In [8]:

# 2. Create `player_num` players with generic names.
players = []
for i in range(player_num):
    players.append(Player(i, get_first_name()))

# 3. Sample the `skill_level` for each player.
skill_levels = np.random.normal(0, 1, player_num)
for i, player in enumerate(players):
    player.skill = skill_levels[i]

metagame_df = pd.read_csv('metagame_spread.csv', header=0, index_col=0)
representation_counts = metagame_df['Representation'].to_dict()

# 2. Create a list of decks based on representation
available_decks = []
for deck_name, count in representation_counts.items():
    available_decks.extend([deck_name] * count)

# Shuffle the list to ensure randomness
np.random.shuffle(available_decks)

# 3. Assign a deck to each player
for i, player in enumerate(players):
    player_deck_name = available_decks[i]
    player.set_deck(deck_manager.decks[player_deck_name])

# 5. Generate the `win_prob_matrix`.
tournament_win_prob_matrix = np.zeros((player_num, player_num))
for i in range(player_num):
    for j in range(player_num):
        if i != j:
            deck_win_prob = players[i].get_deck().get_matchup_win_prob(players[j].get_deck().name)
            skill_adjustment = skill_value_modifier * (players[i].skill - players[j].skill)
            tournament_win_prob_matrix[i, j] = deck_win_prob + skill_adjustment
            tournament_win_prob_matrix[j, i] = 1 - tournament_win_prob_matrix[i, j]

player_names = [f"{str(player.id)} {player.alias} ({player.deck.name}) - Skill: {round(player.skill,3)}" for player in players]

# Pickle the list of players
with open("players.pkl", "wb") as f:
    pickle.dump(players, f)

# Convert to DataFrame for better visualization
pvp_win_prob = pd.DataFrame(tournament_win_prob_matrix, index=player_names, columns=player_names)

pvp_win_prob.to_csv('pvp_win_prob.csv')